In [ ]:
# 필요한 패키지 불러오기
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

# 데이터 로드
train_df = pd.read_csv('train_for_NLP.csv')
test_df = pd.read_csv('test_cleaned.csv')

# train 데이터에서 특성과 레이블 분리
X_train = train_df['combined_str']
y_train = train_df['target']

# test 데이터의 특성
X_test = test_df['combined_str']

# BERT tokenizer 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# 최대 토큰 길이 설정
MAX_LEN = 128

# BERT 임베딩 함수 정의
def encode_texts(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # special tokens [CLS] and [SEP]
            max_length=max_len,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='tf',
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    return np.array(input_ids), np.array(attention_masks)

# Train 데이터 인코딩
X_train_ids, X_train_masks = encode_texts(X_train, tokenizer, MAX_LEN)

# Test 데이터 인코딩
X_test_ids, X_test_masks = encode_texts(X_test, tokenizer, MAX_LEN)

# 데이터를 학습과 검증용으로 분리
X_train_ids_split, X_val_ids_split, y_train_split, y_val_split = train_test_split(X_train_ids, y_train, test_size=0.2, random_state=42)
X_train_masks_split, X_val_masks_split = train_test_split(X_train_masks, test_size=0.2, random_state=42)

# BERT + RNN 모델 정의
def create_rnn_model(bert_model, max_len):
    # BERT 입력 레이어
    input_ids = tf.keras.Input(shape=(max_len,), dtype='int32', name='input_ids')
    attention_masks = tf.keras.Input(shape=(max_len,), dtype='int32', name='attention_mask')

    # BERT에서 출력된 텍스트 임베딩
    bert_output = bert_model(input_ids, attention_mask=attention_masks)[0]

    # RNN 레이어 (여기서는 LSTM 사용)
    x = tf.keras.layers.LSTM(64)(bert_output)

    # Dense 레이어 및 출력 (이진 분류)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # 모델 생성
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output)

    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# 모델 생성
rnn_model = create_rnn_model(bert_model, MAX_LEN)

# 모델 학습
history = rnn_model.fit(
    [X_train_ids_split, X_train_masks_split],
    y_train_split,
    validation_data=([X_val_ids_split, X_val_masks_split], y_val_split),
    epochs=3,
    batch_size=16
)

# 테스트 데이터 예측
y_test_pred_prob_rnn = rnn_model.predict([X_test_ids, X_test_masks]).flatten()
y_test_pred_rnn = (y_test_pred_prob_rnn >= 0.5).astype(int)

# 제출 파일 생성
submission_df = pd.read_csv("sample_submission.csv")
submission_df['target'] = y_test_pred_rnn
submission_df.to_csv("submit_BERT_RNN.csv", index=False)
